# Cosmological Parameter Estimation

***
# Preamble

We will first do some setup that lets us make plots inline in the notebook.
You can run each cell by highlighting it and pressing Shift-Enter.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

***

# Running These Exercises

### MyBinder
You can run these exercises by visting https://mybinder.org/v2/gh/joezuntz/michigan/master but your work will not automatically be saved at the end 
of each session.

**To save your work:**
- In the menu under the Jupyter logo at the top left of the page, select File > Download as > Notebook.  This will save your work to your machine.

**To restore your work:**
- Visit https://mybinder.org/v2/gh/joezuntz/michigan/master again
- Click "Upload"
- Find the file you downloaded and select it
- Click the upload button that appears, and overwrite the existing file.

### Local Jupyter

You can also clone this repository and run it locally using `git clone https://github.com/joezuntz/michigan` if you already know how to do that, but we won't be able to help you with it.

***
# Lecture 1: Essential Data Handling & Modelling

## Exercise 1: Astropy

There are many public cosmology libraries that have been released to help calculate theory predictions or explore data.

One of these is Astropy, a python library for astronomy research.  It contains tools for many common astronomy tasks, including basic (background) cosmology.  It also contains a library for converting and using units, which we will use here too.

In this first exercise we will explore how to create a cosmological model using Astropy, and use it to predict some observable values.

In [ ]:
from astropy.cosmology import LambdaCDM
from astropy.units import mag

When you execute the cell below some documentation will pop up at the bottom.
Read it and learn how to create a new Lambda CDM cosmology model with specified parameters.

HINT: Some parameters are optional.

In [ ]:
LambdaCDM?

Now, create a test model called `cosmo` with Hubble parameter 72.0, matter density 0.3, dark energy density 0.7

In [ ]:
# Write your code here


Your `cosmo` object has functions attached to it ("methods") that calculate different cosmological quantities.
Run the cell below and scroll down in the cell to find a listing of them.

Find the method `distmod` and read about its meaning.

In [ ]:
help(cosmo)

First, use `cosmo.distmod` to work out the distance modulus at the single redshift value $z=1$.

In [ ]:
# Write your code here


Now, create a numpy array of redshifts `z` = 0.1, 0.2, 0.3, ... 1.6 using numpy, and compute the distance modulus to it.
Make a plot of distance modulus versus redshift in this model.

In [ ]:
# Write your code here


Now let's assume that the absolute magnitude of our supernovae is `M0 = -19.3` magnitudes

Use the `distmod` method to calculate the apparent magnitude $m$ of these supernovae, as a function of redshift.

Make a plot of $m$ vs $z$.

HINT: Re-read the help on `distmod` for the relation between apparent and absolute magnitude.  Distmod returns values with units; to remove the units from a quantity `x` you can write `x = x.value`.

In [ ]:
# write your code here


As the final step in this exercise, write a `model` function to predict the apparent magnitude for any set of parameters, for any set of redshifts.

In [ ]:
# Complete this function
def model(H0, Omega_matter, Omega_lambda, M0, z):
    ...

That's the end of lecture 1.  If you've finished it and have time to spare, your stretch goal is to plot a graph of the luminosity distance, angular diameter distance, and comoving distance from z=0 to z=10.

***
## Exercise 2: Loading and exploring supernova data

The *Pantheon* supernova analysis project measured the apparent magnitude of a large sample of supernovae, standardising them to control any differences between them and very carefully calibrating them.  They grouped them into 40 bins of data, which we have extracted and supplied with this notebook.

In this exercise we will load the Pantheon data and explore it.

In later exercises we will use Pantheon to constrain cosmological parameters.

Load in the Pantheon data from the file `data.txt`.  The two columns in it are *redshift* `z_obs` and *apparent magnitude* `m_obs`.

HINT: Use the function `np.loadtxt` to do this quickly, but then you'll need to pull out the two columns separately.

In [ ]:
# write your code here


Make a plot of the data points in the file, showing $m$ as a function of $z$

In [ ]:
#write your code here


Now use the function you wrote in the last exercise to make a theory prediction for $m$ for the same parameters we have used before:
```
    H0 = 72.0
    Omega_matter = 0.3
    Omega_lambda = 0.7
    M0 = -19.3
```
Add it to your plot.
How good is the fit?

In [ ]:
# write your code here


Load the covariance matrix `C` of the Pantheon data points from the file `cov.txt`.  Check its shape is 40 x 40.

In [ ]:
# write your code here.


Display the covariance matrix as an image, and describe what you see.

HINT: `plt.imshow`.
It's easiest to see detail if you plot the log of the absolute value of the matrix.

In [ ]:
# write your code here.


Use the covariance matrix to work out the standard deviation of each data point as an array.

HINT: The covariance $C_{ij}$ between two data points $i$ and $j$ is the expectation $E\left[(x_i - \bar{x_i}) \cdot (x_j - \bar{x_j})\right]$.  The variance of a single data point $i$ is $E\left[(x_i - \bar{x_i})^2\right]$, so think about where the variance is in the covariance matrix. Don't forget the relationship between variance and standard deviation! There are numpy functions that will help with both of these.

In [ ]:
# write your code here.


Make a plot like the first one of $m$ vs $z$, but this time include error bars.

HINT: `plt.errorbar`.  The error bars are very small, so will be hard to see! You could expand them by a factor of 10.

In [ ]:
# write your code here.


Compute the inverse `invC` of the matrix `C` and display it in the same way as you did the covariance.

HINT: `np.linalg.inv`

In [ ]:
# write your code here.


For this set of parameters, compute the log-likelihood:

$-\frac{1}{2} * (m_\textrm{obs} - m_\textrm{pred})^T \cdot C^{-1} \cdot(m_\textrm{obs} - m_\textrm{pred})$

Roughly speaking a good log-likelihood should be about $-\frac{1}{2} n_\mathrm{data}$.  Is this a good fit?


HINT: Use the $@$ sign to do the The matrix (and dot) product in numpy

In [ ]:
# write your code here


Finally, write a function `loglike` that takes a vector of your four parameters, `[H0, Omega_matter, Omega_de, M0]`, and computes their likelihood given the observed data.

Astropy will not work if a negative value of Omega_matter or H0 is supplied. Make your function check for this and return -infinity if this happens.

In [ ]:
# complete this function
def loglike(p):
    ...


# Lecture 2: Exploring Likelihoods

***
## Exercise 3: Exploring the likelihood on a grid

We will now explore the likelihood while fixing two of the parameters, `Omega_matter=0.3` and `Omega_lambda=0.7`, and just varying `H0` and `M0`.

Compute your log-likelihood on a grid of 100 values each of `H0 = 65 .. 80` and `M0 = -19.3 .. 19.1`.

This may take up to a minute - use a timer (e.g. on your phone) to measure how long it takes.

HINT: You can use `np.linspace` to generate the set of values for each parameters.  You will need to make two loops, one through each pair of values.

In [ ]:
# write your code here
N = 100


Display an image of the likelihood (not the log-likelihood).

Describe what you see, and its mathematical and physical meaning.

HINT: You can use the `extent` argument to `plt.imshow` to get the axes right, and the option `aspect='auto'`

In [ ]:
# write your code here


Imagine you had to explore the other two parameters, `Omega_matter` and `Omega_lambda`, at the same time as these two parameters, using the same method.

Estimate how long in days this would take to calculate.


In [ ]:
# write your code here


# Exercise 4: Sampling

In the last exercise you used a brute-force grid search to explore a 2D parameter space of a model for the Pantheon supernovae.

You should have found that it would take several days to extend this to a 4D grid, which in unreasonably long, though feasible.  Models for other cosmology data sets might have more than 30 parameters, which would take longer than the lifetime of the Universe to explore like this.

We will use the Metropolis-Hastings algorithm that we described in the lectures to explore this parameter space instead, to make it take a manageable time.

We will first need a proposal function, which should returns a random jump from a current position in parameter space to one that is nearby.

Write a proposal that adds a small Gaussian-distributed jump from the current point `p` in each direction.  Since the parameters are all different they need different sized jumps - you should make the typical size be these values for the different parameters:

```
 H0: 0.5
 omega_matter: 0.01
 omega_lambda: 0.01
 M0: 0.025
```

HINT: `np.random.normal(n)` generates `n` Gaussian-distributed random numbers with standard deviation 1.

In [ ]:
# complete this function
def propose(p):
    ...

Now we will write a Metropolis-Hastings MCMC sampling process.

Write a code to perform the MH loop that we described in lectures, with these features:
- record the value of the new parameter set each time through the loop.
- record the likelihood each iteration.
- do 100,000 iterations of the loop
- start from the point `[72., 0.3, 0.7, -19.3]`
- count the number of time the proposal is accepted.  If everything is working, about 166% of the jumps should be accepted in this case.

This will take a few minutes - you might want to print something out every 5000 samples to give you an idea of progress.

HINT: It's easiest to pre-allocate the arrays of parameter values and likelihoods. It's also easier always to work in log-likelihoods.  This will take a few minutes to run.

In [ ]:
# write your code here


Plot the value of each parameter throughout the chain.  Comment on the performance of the proposal.

Why are some parameters better explored than others?  How could you improve the performance?


In [ ]:
# write your code here


Make a point plot of H0 versus M0 across the chain.

Compare this to your grid plot earlier.

HINT: Use a comma `','` as the marker type to make it easier to see.

In [ ]:
# write your code here


Make a scatter plot of omega_matter versus omega_lambda for the full chain.

Use the likelihood value as the color of each point, and size=1 for each.

HINT: Use `plt.scatter`


In [ ]:
# write your code here


Make a histogram of each parameter across the chain..

Use the log-likelihood value as the color of each point, and size `s=1` for each.

HINT: Use `plt.hist`

In [ ]:
# write your code here


Compute the mean and standard deviation of Omega_matter and Omega_lambda, across the chain.
This estimate is what this data set tells us about these parameters.

In [ ]:
# write your code here


You can also work out derived parameters from the chain.

Plot a histogram of $\Omega_k = 1 - \Omega_m - \Omega_\lambda$, and compute its mean and standard deviation

In [ ]:
# write your code here


# Further exercises

- Use the library `emcee` instead of your own metropolis sampler to 
- Improve your Metropolis-Hastings sampler by:
    - taking the posterior covariance of the transpose of your existing chain
    - finding the Cholesky decomposition `chol` (matrix square-root) of the posterior covariance
    - making a new proposal `chol @ r` where `r` is a vector of random Gaussian numbers with variance 1

